Image Extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os

In [43]:
def save_images(filename, path):
    if not os.path.exists(path):
        os.makedirs(path)

    vidcap = cv2.VideoCapture(path + "//" + filename)

    # get the number of frames
    n_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    # saved frames
    # saved_frames = [0, int(n_frames/3), int(2*n_frames/3), n_frames-1]
    saved_frames = [0, 5,  int(n_frames/3), int(n_frames/2), int(2*n_frames/3), n_frames-1]


    # save the first frame
    for i in range(0, 6):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, saved_frames[i])
        success, image = vidcap.read()
        dir = None
        image_name = None
        if i == 0 or i == 1 or i== 5:
            dir = os.path.join(path, "..\\images3", filename[0:3], 'neutral').replace("\\","/")
            image_name = filename[0:5]+filename[-5]+'_neutral_'+str(saved_frames[i])+'.jpg'
        else:
            dir = os.path.join(path, "..\\images3", filename[0:3], 'smile').replace("\\","/")
            image_name = filename[0:5]+filename[-5]+'_smile_'+str(saved_frames[i])+'.jpg'

        if not os.path.exists(dir):
            os.makedirs(dir)


        name = os.path.join(dir, image_name).replace("\\","/")
        # print('image', i, 'saved at', name)
        cv2.imwrite(name, image)


In [ ]:
directory = '/content/drive/MyDrive/Msthesis/Dataset/videos'
count = 1

# loop for all video files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".mp4") and filename.__contains__("spontaneous"):
        # save images
        print('\rcount:', count, ', Saving images for', filename, '             ', end='')
        save_images(filename, directory)
        count = count + 1
    # if count > 1:
    #   break

count: 597 , Saving images for 464_spontaneous_smile_4.mp4              

Create Landmark files

Importing Library

In [44]:
#loads data from dataset with a path
import os
import cv2


#landmarks
from imutils import face_utils
import argparse
import imutils
import dlib

Input dataset location

In [ ]:
dataset_path = '/content/drive/MyDrive/Msthesis/Dataset/images3'
print(dataset_path)

/content/drive/MyDrive/Msthesis/Dataset/images3


define locations to store landmarks file

In [ ]:
processed_data_path = dataset_path+"/../processed_data3"
print(processed_data_path)


if not os.path.exists(processed_data_path):
    os.mkdir(processed_data_path)



landmark_path = processed_data_path + "/landmarks/"
if not os.path.exists(landmark_path):
    os.mkdir(landmark_path)

annotated_image_path =  processed_data_path + "/annotated_images/"
if not os.path.exists(annotated_image_path):
    os.mkdir(annotated_image_path)


print("landmark_path:", landmark_path)
print("annotated_image_path:", annotated_image_path)

/content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3
landmark_path: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/
annotated_image_path: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/annotated_images/


Initializing the detector and predictor for landmark detection

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./drive/MyDrive/Msthesis/code/shape_predictor_68_face_landmarks.dat")

Data loader Class

In [45]:
class data_loader():
    def load_data(self, path, category='smile'):
        #for each subfolder(labels) in the path
        for label in os.listdir(path):
            #for each image in the subfolder
            if not os.path.exists(landmark_path + label):
              os.mkdir(landmark_path + label)

            if not os.path.exists(annotated_image_path + label):
              os.mkdir(annotated_image_path + label)

            for image_name in os.listdir(path + label+"/"+category):

                print("the path created:", landmark_path + label+"/"+category)
                if not os.path.exists(landmark_path + label+"/"+category):
                    os.mkdir(landmark_path + label+"/"+category)
                if not os.path.exists(annotated_image_path + label+"/"+category):
                    os.mkdir(annotated_image_path + label+"/"+category)


                #load the image
                image = cv2.imread(path + label + "/"+category+"/" + image_name)

                #get 2D landmarks from the image with the annoted image
                landmarks,image = self.get_2D_landmarks(image)

                #save the landmarks in a file
                file_landmarks = open(landmark_path + label + "/" + category + "/" + image_name[:-4] + ".txt", "w")
                for landmark in landmarks:
                    file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

                #save the annotated image
                cv2.imwrite(annotated_image_path + label + "/"+ category + "/" + image_name, image)

            print("label:", label)





    def get_2D_landmarks(self, image):
        #get 2D landmarks from the image

        #resize image to a fixed-size
        image =imutils.resize(image, width=500)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)

        landmarks = []
        for (i, rect) in enumerate(rects):
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            # convert dlib's rectangle to a OpenCV-style bounding box
            # [i.e., (x, y, w, h)], then draw the face bounding box
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # show the face number
            cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw them on the image
            for (x, y) in shape:
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
                landmarks.append((x, y))

            #annotate the image with the landmarks

        return landmarks,image

In [ ]:
dl = data_loader()
print('Loading neutral images...')
dl.load_data(dataset_path+"/", category='neutral')
print('Neutral images loading done!\nLoading smile images...')
dl.load_data(dataset_path+"/", category='smile')
print('Smile images loading done!')

Loading neutral images...
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/001/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/001/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/001/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/001/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/001/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/001/neutral
label: 001
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/013/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/013/neutral
the path created: /content/drive/MyDrive/Msthesis/Dataset/images3/../processed_data3/landmarks/013/neutral


Feature Extractor

Mouth_features

In [46]:
import numpy as np

class Features_Mouth:

    def normalize_array(self, arr):
        if len(arr) == 0:
            return arr
        arr = np.array(arr)
        max = arr.max()
        if max == 0:
            return arr
        arr = arr / max
        return arr.tolist()

    def get_features_from_path(self, image_path, includes='dabenm'):

        # landmark_file_path = landmark_path + label + "/" + cat + "/" +landmark_file_name
        image = cv2.imread(image_path)


        dl = data_loader()
        ldm, _ = dl.get_2D_landmarks(image)

        print("length:", len(ldm))

         #save the landmarks in a file
        file_landmarks = open("temp.txt", "w")
        for landmark in ldm:
            file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

        file_landmarks.close()

        file_landmarks = open("temp.txt", "r")
        lines = file_landmarks.readlines()

        print("lines:", lines)

        features = self.get_features(lines, includes)

        print("features: ", features)

        return features


    def get_features(self, lines, includes="dabenm"):
        self.includes = includes
        # function to get the distance features from the landmarks
        #convert the lines to a list of tuples
        lines = [tuple(map(int, line.split())) for line in lines]


        def distance(a,b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def get_distance_features(lines):

            def slope(a,b):
                return ((b[1]-a[1])/(b[0]-a[0]))

            # pairs from landmark_based_fr.pdf

            basic_shape_pairs = [
                [lines[1], lines[15]],
                [lines[27], lines[57]],
                [lines[36], lines[48]],
                [lines[45], lines[54]],
                [lines[36], lines[45]],
                [lines[17], lines[21]],
                [lines[22], lines[26]],
            ]

            eyes_pairs = [
                [lines[36], lines[39]],
                [lines[42], lines[45]],
                [lines[38], lines[40]],
                [lines[43], lines[47]],
            ]

            nose_pairs = [
                [lines[31], lines[35]],
                [lines[27], lines[30]],
                [lines[30], lines[31]],
                [lines[30], lines[35]],
                [lines[28], lines[31]],
                [lines[28], lines[35]],
            ]

            mouth_pairs = [
                [lines[48], lines[54]],
                [lines[60], lines[64]],
                [lines[49], lines[53]],
                [lines[59], lines[55]],
            ]

            basic_shape_dists = [distance(pair[0], pair[1]) for pair in basic_shape_pairs]
            nose_dists = [distance(pair[0], pair[1]) for pair in nose_pairs]
            mouth_dists = [distance(pair[0], pair[1]) for pair in mouth_pairs]
            eyes_dists = [distance(pair[0], pair[1]) for pair in eyes_pairs]



            features = []
            if "b" in self.includes:
                features = features + basic_shape_dists
            if "m" in self.includes:
                features = features + mouth_dists
            if "n" in self.includes:
                features = features + nose_dists
            if "e" in self.includes:
                features = features + eyes_dists

            if 'd' in self.includes:
                features = features
            else:
                features = basic_shape_dists

            features = self.normalize_array(features)

            return features

        # function to get the angle features from the landmarks
        def get_angle_features(self, lines):
            def angle(a,b,c):
                a = np.asarray(a)
                b = np.asarray(b)
                c = np.asarray(c)
                ba = a - b
                bc = c - b

                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                if cosine_angle < -1 or cosine_angle > 1:
                    cosine_angle = 0.8
                angle = np.arccos(cosine_angle)

                return angle

            # triplets to find the angle.
            mouth_triplets = [
                [lines[50], lines[48], lines[61]],
                [lines[67], lines[48], lines[58]],
                [lines[52], lines[54], lines[63]],
                [lines[65], lines[54], lines[56]],
            ]

            eyes_triplets = [
                [lines[37], lines[36], lines[41]],
                [lines[38], lines[39], lines[40]],
                [lines[43], lines[42], lines[47]],
                [lines[44], lines[45], lines[46]],
            ]

            nose_triplets = [
                [lines[27], lines[31], lines[35]],
                [lines[30], lines[31], lines[35]],
                [lines[27], lines[35], lines[31]],
                [lines[30], lines[35], lines[31]],
            ]

            # angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in point_triplets]

            mouth_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in mouth_triplets]
            eyes_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in eyes_triplets]
            eyes_angle_features = [0,0,0,0]
            nose_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in nose_triplets]
            nose_angle_features = [0,0,0,0]

            angle_features = []

            if "m" in self.includes:
                angle_features = angle_features + mouth_angle_features
            if "n" in self.includes:
                angle_features = angle_features + nose_angle_features
            if "e" in self.includes:
                angle_features = angle_features + eyes_angle_features


            angle_features = self.normalize_array(angle_features)

            return angle_features

        # get distance features
        dist_features = get_distance_features(lines)

        # get angle features
        angle_features = get_angle_features(self, lines)

        # combine the features
        # features = dist_features + angle_features
        # features = dist_features
        # print("features:", features)

        features = dist_features

        # if 'd' in self.includes:
        #     features = features + dist_features
        if 'a' in self.includes:
            features = features + angle_features

        return features

Nose_features

In [47]:
import numpy as np

class Features_Nose:

    def normalize_array(self, arr):
        if len(arr) == 0:
            return arr
        arr = np.array(arr)
        max = arr.max()
        if max == 0:
            return arr
        arr = arr / max
        return arr.tolist()

    def get_features_from_path(self, image_path, includes='dabenm'):

        # landmark_file_path = landmark_path + label + "/" + cat + "/" +landmark_file_name
        image = cv2.imread(image_path)


        dl = data_loader()
        ldm, _ = dl.get_2D_landmarks(image)

        print("length:", len(ldm))

         #save the landmarks in a file
        file_landmarks = open("temp.txt", "w")
        for landmark in ldm:
            file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

        file_landmarks.close()

        file_landmarks = open("temp.txt", "r")
        lines = file_landmarks.readlines()

        print("lines:", lines)

        features = self.get_features(lines, includes)

        print("features: ", features)

        return features


    def get_features(self, lines, includes="dabenm"):
        self.includes = includes
        # function to get the distance features from the landmarks
        #convert the lines to a list of tuples
        lines = [tuple(map(int, line.split())) for line in lines]


        def distance(a,b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def get_distance_features(lines):

            def slope(a,b):
                return ((b[1]-a[1])/(b[0]-a[0]))

            # pairs from landmark_based_fr.pdf

            basic_shape_pairs = [
                [lines[1], lines[15]],
                [lines[27], lines[57]],
                [lines[36], lines[48]],
                [lines[45], lines[54]],
                [lines[36], lines[45]],
                [lines[17], lines[21]],
                [lines[22], lines[26]],
            ]

            eyes_pairs = [
                [lines[36], lines[39]],
                [lines[42], lines[45]],
                [lines[38], lines[40]],
                [lines[43], lines[47]],
            ]

            nose_pairs = [
                [lines[31], lines[35]],
                [lines[27], lines[30]],
                [lines[30], lines[31]],
                [lines[30], lines[35]],
                [lines[28], lines[31]],
                [lines[28], lines[35]],
            ]

            mouth_pairs = [
                [lines[48], lines[54]],
                [lines[60], lines[64]],
                [lines[49], lines[53]],
                [lines[59], lines[55]],
            ]

            basic_shape_dists = [distance(pair[0], pair[1]) for pair in basic_shape_pairs]
            nose_dists = [distance(pair[0], pair[1]) for pair in nose_pairs]
            mouth_dists = [distance(pair[0], pair[1]) for pair in mouth_pairs]
            eyes_dists = [distance(pair[0], pair[1]) for pair in eyes_pairs]



            features = []
            if "b" in self.includes:
                features = features + basic_shape_dists
            if "m" in self.includes:
                features = features + mouth_dists
            if "n" in self.includes:
                features = features + nose_dists
            if "e" in self.includes:
                features = features + eyes_dists

            if 'd' in self.includes:
                features = features
            else:
                features = basic_shape_dists

            features = self.normalize_array(features)

            return features

        # function to get the angle features from the landmarks
        def get_angle_features(self, lines):
            def angle(a,b,c):
                a = np.asarray(a)
                b = np.asarray(b)
                c = np.asarray(c)
                ba = a - b
                bc = c - b

                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                if cosine_angle < -1 or cosine_angle > 1:
                    cosine_angle = 0.8
                angle = np.arccos(cosine_angle)

                return angle

            # triplets to find the angle.
            mouth_triplets = [
                [lines[50], lines[48], lines[61]],
                [lines[67], lines[48], lines[58]],
                [lines[52], lines[54], lines[63]],
                [lines[65], lines[54], lines[56]],
            ]

            eyes_triplets = [
                [lines[37], lines[36], lines[41]],
                [lines[38], lines[39], lines[40]],
                [lines[43], lines[42], lines[47]],
                [lines[44], lines[45], lines[46]],
            ]

            nose_triplets = [
                [lines[27], lines[31], lines[35]],
                [lines[30], lines[31], lines[35]],
                [lines[27], lines[35], lines[31]],
                [lines[30], lines[35], lines[31]],
            ]

            # angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in point_triplets]

            mouth_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in mouth_triplets]
            mouth_angle_features = [0,0,0,0]
            eyes_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in eyes_triplets]
            eyes_angle_features = [0,0,0,0]
            nose_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in nose_triplets]


            angle_features = []

            if "m" in self.includes:
                angle_features = angle_features + mouth_angle_features
            if "n" in self.includes:
                angle_features = angle_features + nose_angle_features
            if "e" in self.includes:
                angle_features = angle_features + eyes_angle_features


            angle_features = self.normalize_array(angle_features)

            return angle_features

        # get distance features
        dist_features = get_distance_features(lines)

        # get angle features
        angle_features = get_angle_features(self, lines)

        # combine the features
        # features = dist_features + angle_features
        # features = dist_features
        # print("features:", features)

        features = dist_features

        # if 'd' in self.includes:
        #     features = features + dist_features
        if 'a' in self.includes:
            features = features + angle_features

        return features

Eye_Features

In [48]:
import numpy as np

class Features_Eye:

    def normalize_array(self, arr):
        if len(arr) == 0:
            return arr
        arr = np.array(arr)
        max = arr.max()
        if max == 0:
            return arr
        arr = arr / max
        return arr.tolist()

    def get_features_from_path(self, image_path, includes='dabenm'):

        # landmark_file_path = landmark_path + label + "/" + cat + "/" +landmark_file_name
        image = cv2.imread(image_path)


        dl = data_loader()
        ldm, _ = dl.get_2D_landmarks(image)

        print("length:", len(ldm))

         #save the landmarks in a file
        file_landmarks = open("temp.txt", "w")
        for landmark in ldm:
            file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

        file_landmarks.close()

        file_landmarks = open("temp.txt", "r")
        lines = file_landmarks.readlines()

        print("lines:", lines)

        features = self.get_features(lines, includes)

        print("features: ", features)

        return features


    def get_features(self, lines, includes="dabenm"):
        self.includes = includes
        # function to get the distance features from the landmarks
        #convert the lines to a list of tuples
        lines = [tuple(map(int, line.split())) for line in lines]


        def distance(a,b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def get_distance_features(lines):

            def slope(a,b):
                return ((b[1]-a[1])/(b[0]-a[0]))

            # pairs from landmark_based_fr.pdf

            basic_shape_pairs = [
                [lines[1], lines[15]],
                [lines[27], lines[57]],
                [lines[36], lines[48]],
                [lines[45], lines[54]],
                [lines[36], lines[45]],
                [lines[17], lines[21]],
                [lines[22], lines[26]],
            ]

            eyes_pairs = [
                [lines[36], lines[39]],
                [lines[42], lines[45]],
                [lines[38], lines[40]],
                [lines[43], lines[47]],
            ]

            nose_pairs = [
                [lines[31], lines[35]],
                [lines[27], lines[30]],
                [lines[30], lines[31]],
                [lines[30], lines[35]],
                [lines[28], lines[31]],
                [lines[28], lines[35]],
            ]

            mouth_pairs = [
                [lines[48], lines[54]],
                [lines[60], lines[64]],
                [lines[49], lines[53]],
                [lines[59], lines[55]],
            ]

            basic_shape_dists = [distance(pair[0], pair[1]) for pair in basic_shape_pairs]
            nose_dists = [distance(pair[0], pair[1]) for pair in nose_pairs]
            mouth_dists = [distance(pair[0], pair[1]) for pair in mouth_pairs]
            eyes_dists = [distance(pair[0], pair[1]) for pair in eyes_pairs]



            features = []
            if "b" in self.includes:
                features = features + basic_shape_dists
            if "m" in self.includes:
                features = features + mouth_dists
            if "n" in self.includes:
                features = features + nose_dists
            if "e" in self.includes:
                features = features + eyes_dists

            if 'd' in self.includes:
                features = features
            else:
                features = basic_shape_dists

            features = self.normalize_array(features)

            return features

        # function to get the angle features from the landmarks
        def get_angle_features(self, lines):
            def angle(a,b,c):
                a = np.asarray(a)
                b = np.asarray(b)
                c = np.asarray(c)
                ba = a - b
                bc = c - b

                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                if cosine_angle < -1 or cosine_angle > 1:
                    cosine_angle = 0.8
                angle = np.arccos(cosine_angle)

                return angle

            # triplets to find the angle.
            mouth_triplets = [
                [lines[50], lines[48], lines[61]],
                [lines[67], lines[48], lines[58]],
                [lines[52], lines[54], lines[63]],
                [lines[65], lines[54], lines[56]],
            ]

            eyes_triplets = [
                [lines[37], lines[36], lines[41]],
                [lines[38], lines[39], lines[40]],
                [lines[43], lines[42], lines[47]],
                [lines[44], lines[45], lines[46]],
            ]

            nose_triplets = [
                [lines[27], lines[31], lines[35]],
                [lines[30], lines[31], lines[35]],
                [lines[27], lines[35], lines[31]],
                [lines[30], lines[35], lines[31]],
            ]

            # angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in point_triplets]

            mouth_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in mouth_triplets]
            mouth_angle_features = [0,0,0,0]
            eyes_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in eyes_triplets]
            nose_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in nose_triplets]
            nose_angle_features = [0,0,0,0]


            angle_features = []

            if "m" in self.includes:
                angle_features = angle_features + mouth_angle_features
            if "n" in self.includes:
                angle_features = angle_features + nose_angle_features
            if "e" in self.includes:
                angle_features = angle_features + eyes_angle_features


            angle_features = self.normalize_array(angle_features)

            return angle_features

        # get distance features
        dist_features = get_distance_features(lines)

        # get angle features
        angle_features = get_angle_features(self, lines)

        # combine the features
        # features = dist_features + angle_features
        # features = dist_features
        # print("features:", features)

        features = dist_features

        # if 'd' in self.includes:
        #     features = features + dist_features
        if 'a' in self.includes:
            features = features + angle_features

        return features

Mouth-Eye features

In [49]:
import numpy as np

class Features_mouth_eye:

    def normalize_array(self, arr):
        if len(arr) == 0:
            return arr
        arr = np.array(arr)
        max = arr.max()
        if max == 0:
            return arr
        arr = arr / max
        return arr.tolist()

    def get_features_from_path(self, image_path, includes='dabenm'):

        # landmark_file_path = landmark_path + label + "/" + cat + "/" +landmark_file_name
        image = cv2.imread(image_path)


        dl = data_loader()
        ldm, _ = dl.get_2D_landmarks(image)

        print("length:", len(ldm))

         #save the landmarks in a file
        file_landmarks = open("temp.txt", "w")
        for landmark in ldm:
            file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

        file_landmarks.close()

        file_landmarks = open("temp.txt", "r")
        lines = file_landmarks.readlines()

        print("lines:", lines)

        features = self.get_features(lines, includes)

        print("features: ", features)

        return features


    def get_features(self, lines, includes="dabenm"):
        self.includes = includes
        # function to get the distance features from the landmarks
        #convert the lines to a list of tuples
        lines = [tuple(map(int, line.split())) for line in lines]


        def distance(a,b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def get_distance_features(lines):

            def slope(a,b):
                return ((b[1]-a[1])/(b[0]-a[0]))

            # pairs from landmark_based_fr.pdf

            basic_shape_pairs = [
                [lines[1], lines[15]],
                [lines[27], lines[57]],
                [lines[36], lines[48]],
                [lines[45], lines[54]],
                [lines[36], lines[45]],
                [lines[17], lines[21]],
                [lines[22], lines[26]],
            ]

            eyes_pairs = [
                [lines[36], lines[39]],
                [lines[42], lines[45]],
                [lines[38], lines[40]],
                [lines[43], lines[47]],
            ]

            nose_pairs = [
                [lines[31], lines[35]],
                [lines[27], lines[30]],
                [lines[30], lines[31]],
                [lines[30], lines[35]],
                [lines[28], lines[31]],
                [lines[28], lines[35]],
            ]

            mouth_pairs = [
                [lines[48], lines[54]],
                [lines[60], lines[64]],
                [lines[49], lines[53]],
                [lines[59], lines[55]],
            ]

            basic_shape_dists = [distance(pair[0], pair[1]) for pair in basic_shape_pairs]
            nose_dists = [distance(pair[0], pair[1]) for pair in nose_pairs]
            mouth_dists = [distance(pair[0], pair[1]) for pair in mouth_pairs]
            eyes_dists = [distance(pair[0], pair[1]) for pair in eyes_pairs]



            features = []
            if "b" in self.includes:
                features = features + basic_shape_dists
            if "m" in self.includes:
                features = features + mouth_dists
            if "n" in self.includes:
                features = features + nose_dists
            if "e" in self.includes:
                features = features + eyes_dists

            if 'd' in self.includes:
                features = features
            else:
                features = basic_shape_dists

            features = self.normalize_array(features)

            return features

        # function to get the angle features from the landmarks
        def get_angle_features(self, lines):
            def angle(a,b,c):
                a = np.asarray(a)
                b = np.asarray(b)
                c = np.asarray(c)
                ba = a - b
                bc = c - b

                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                if cosine_angle < -1 or cosine_angle > 1:
                    cosine_angle = 0.8
                angle = np.arccos(cosine_angle)

                return angle

            # triplets to find the angle.
            mouth_triplets = [
                [lines[50], lines[48], lines[61]],
                [lines[67], lines[48], lines[58]],
                [lines[52], lines[54], lines[63]],
                [lines[65], lines[54], lines[56]],
            ]

            eyes_triplets = [
                [lines[37], lines[36], lines[41]],
                [lines[38], lines[39], lines[40]],
                [lines[43], lines[42], lines[47]],
                [lines[44], lines[45], lines[46]],
            ]

            nose_triplets = [
                [lines[27], lines[31], lines[35]],
                [lines[30], lines[31], lines[35]],
                [lines[27], lines[35], lines[31]],
                [lines[30], lines[35], lines[31]],
            ]

            # angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in point_triplets]

            mouth_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in mouth_triplets]
            eyes_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in eyes_triplets]
            nose_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in nose_triplets]
            nose_angle_features = [0,0,0,0]


            angle_features = []

            if "m" in self.includes:
                angle_features = angle_features + mouth_angle_features
            if "n" in self.includes:
                angle_features = angle_features + nose_angle_features
            if "e" in self.includes:
                angle_features = angle_features + eyes_angle_features


            angle_features = self.normalize_array(angle_features)

            return angle_features

        # get distance features
        dist_features = get_distance_features(lines)

        # get angle features
        angle_features = get_angle_features(self, lines)

        # combine the features
        # features = dist_features + angle_features
        # features = dist_features
        # print("features:", features)

        features = dist_features

        # if 'd' in self.includes:
        #     features = features + dist_features
        if 'a' in self.includes:
            features = features + angle_features

        return features

Eye-Nose Features

In [50]:
import numpy as np

class Features_eye_nose:

    def normalize_array(self, arr):
        if len(arr) == 0:
            return arr
        arr = np.array(arr)
        max = arr.max()
        if max == 0:
            return arr
        arr = arr / max
        return arr.tolist()

    def get_features_from_path(self, image_path, includes='dabenm'):

        # landmark_file_path = landmark_path + label + "/" + cat + "/" +landmark_file_name
        image = cv2.imread(image_path)


        dl = data_loader()
        ldm, _ = dl.get_2D_landmarks(image)

        print("length:", len(ldm))

         #save the landmarks in a file
        file_landmarks = open("temp.txt", "w")
        for landmark in ldm:
            file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

        file_landmarks.close()

        file_landmarks = open("temp.txt", "r")
        lines = file_landmarks.readlines()

        print("lines:", lines)

        features = self.get_features(lines, includes)

        print("features: ", features)

        return features


    def get_features(self, lines, includes="dabenm"):
        self.includes = includes
        # function to get the distance features from the landmarks
        #convert the lines to a list of tuples
        lines = [tuple(map(int, line.split())) for line in lines]


        def distance(a,b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def get_distance_features(lines):

            def slope(a,b):
                return ((b[1]-a[1])/(b[0]-a[0]))

            # pairs from landmark_based_fr.pdf

            basic_shape_pairs = [
                [lines[1], lines[15]],
                [lines[27], lines[57]],
                [lines[36], lines[48]],
                [lines[45], lines[54]],
                [lines[36], lines[45]],
                [lines[17], lines[21]],
                [lines[22], lines[26]],
            ]

            eyes_pairs = [
                [lines[36], lines[39]],
                [lines[42], lines[45]],
                [lines[38], lines[40]],
                [lines[43], lines[47]],
            ]

            nose_pairs = [
                [lines[31], lines[35]],
                [lines[27], lines[30]],
                [lines[30], lines[31]],
                [lines[30], lines[35]],
                [lines[28], lines[31]],
                [lines[28], lines[35]],
            ]

            mouth_pairs = [
                [lines[48], lines[54]],
                [lines[60], lines[64]],
                [lines[49], lines[53]],
                [lines[59], lines[55]],
            ]

            basic_shape_dists = [distance(pair[0], pair[1]) for pair in basic_shape_pairs]
            nose_dists = [distance(pair[0], pair[1]) for pair in nose_pairs]
            mouth_dists = [distance(pair[0], pair[1]) for pair in mouth_pairs]
            eyes_dists = [distance(pair[0], pair[1]) for pair in eyes_pairs]



            features = []
            if "b" in self.includes:
                features = features + basic_shape_dists
            if "m" in self.includes:
                features = features + mouth_dists
            if "n" in self.includes:
                features = features + nose_dists
            if "e" in self.includes:
                features = features + eyes_dists

            if 'd' in self.includes:
                features = features
            else:
                features = basic_shape_dists

            features = self.normalize_array(features)

            return features

        # function to get the angle features from the landmarks
        def get_angle_features(self, lines):
            def angle(a,b,c):
                a = np.asarray(a)
                b = np.asarray(b)
                c = np.asarray(c)
                ba = a - b
                bc = c - b

                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                if cosine_angle < -1 or cosine_angle > 1:
                    cosine_angle = 0.8
                angle = np.arccos(cosine_angle)

                return angle

            # triplets to find the angle.
            mouth_triplets = [
                [lines[50], lines[48], lines[61]],
                [lines[67], lines[48], lines[58]],
                [lines[52], lines[54], lines[63]],
                [lines[65], lines[54], lines[56]],
            ]

            eyes_triplets = [
                [lines[37], lines[36], lines[41]],
                [lines[38], lines[39], lines[40]],
                [lines[43], lines[42], lines[47]],
                [lines[44], lines[45], lines[46]],
            ]

            nose_triplets = [
                [lines[27], lines[31], lines[35]],
                [lines[30], lines[31], lines[35]],
                [lines[27], lines[35], lines[31]],
                [lines[30], lines[35], lines[31]],
            ]

            # angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in point_triplets]

            mouth_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in mouth_triplets]
            mouth_angle_features = [0,0,0,0]
            eyes_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in eyes_triplets]
            nose_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in nose_triplets]



            angle_features = []

            if "m" in self.includes:
                angle_features = angle_features + mouth_angle_features
            if "n" in self.includes:
                angle_features = angle_features + nose_angle_features
            if "e" in self.includes:
                angle_features = angle_features + eyes_angle_features


            angle_features = self.normalize_array(angle_features)

            return angle_features

        # get distance features
        dist_features = get_distance_features(lines)

        # get angle features
        angle_features = get_angle_features(self, lines)

        # combine the features
        # features = dist_features + angle_features
        # features = dist_features
        # print("features:", features)

        features = dist_features

        # if 'd' in self.includes:
        #     features = features + dist_features
        if 'a' in self.includes:
            features = features + angle_features

        return features

Mouth-Nose Features




In [51]:
import numpy as np

class Features_mouth_nose:

    def normalize_array(self, arr):
        if len(arr) == 0:
            return arr
        arr = np.array(arr)
        max = arr.max()
        if max == 0:
            return arr
        arr = arr / max
        return arr.tolist()

    def get_features_from_path(self, image_path, includes='dabenm'):

        # landmark_file_path = landmark_path + label + "/" + cat + "/" +landmark_file_name
        image = cv2.imread(image_path)


        dl = data_loader()
        ldm, _ = dl.get_2D_landmarks(image)

        print("length:", len(ldm))

         #save the landmarks in a file
        file_landmarks = open("temp.txt", "w")
        for landmark in ldm:
            file_landmarks.write(str(landmark[0]) + " " + str(landmark[1]) + "\n")

        file_landmarks.close()

        file_landmarks = open("temp.txt", "r")
        lines = file_landmarks.readlines()

        print("lines:", lines)

        features = self.get_features(lines, includes)

        print("features: ", features)

        return features


    def get_features(self, lines, includes="dabenm"):
        self.includes = includes
        # function to get the distance features from the landmarks
        #convert the lines to a list of tuples
        lines = [tuple(map(int, line.split())) for line in lines]


        def distance(a,b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def get_distance_features(lines):

            def slope(a,b):
                return ((b[1]-a[1])/(b[0]-a[0]))

            # pairs from landmark_based_fr.pdf

            basic_shape_pairs = [
                [lines[1], lines[15]],
                [lines[27], lines[57]],
                [lines[36], lines[48]],
                [lines[45], lines[54]],
                [lines[36], lines[45]],
                [lines[17], lines[21]],
                [lines[22], lines[26]],
            ]

            eyes_pairs = [
                [lines[36], lines[39]],
                [lines[42], lines[45]],
                [lines[38], lines[40]],
                [lines[43], lines[47]],
            ]

            nose_pairs = [
                [lines[31], lines[35]],
                [lines[27], lines[30]],
                [lines[30], lines[31]],
                [lines[30], lines[35]],
                [lines[28], lines[31]],
                [lines[28], lines[35]],
            ]

            mouth_pairs = [
                [lines[48], lines[54]],
                [lines[60], lines[64]],
                [lines[49], lines[53]],
                [lines[59], lines[55]],
            ]

            basic_shape_dists = [distance(pair[0], pair[1]) for pair in basic_shape_pairs]
            nose_dists = [distance(pair[0], pair[1]) for pair in nose_pairs]
            mouth_dists = [distance(pair[0], pair[1]) for pair in mouth_pairs]
            eyes_dists = [distance(pair[0], pair[1]) for pair in eyes_pairs]



            features = []
            if "b" in self.includes:
                features = features + basic_shape_dists
            if "m" in self.includes:
                features = features + mouth_dists
            if "n" in self.includes:
                features = features + nose_dists
            if "e" in self.includes:
                features = features + eyes_dists

            if 'd' in self.includes:
                features = features
            else:
                features = basic_shape_dists

            features = self.normalize_array(features)

            return features

        # function to get the angle features from the landmarks
        def get_angle_features(self, lines):
            def angle(a,b,c):
                a = np.asarray(a)
                b = np.asarray(b)
                c = np.asarray(c)
                ba = a - b
                bc = c - b

                cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                if cosine_angle < -1 or cosine_angle > 1:
                    cosine_angle = 0.8
                angle = np.arccos(cosine_angle)

                return angle

            # triplets to find the angle.
            mouth_triplets = [
                [lines[50], lines[48], lines[61]],
                [lines[67], lines[48], lines[58]],
                [lines[52], lines[54], lines[63]],
                [lines[65], lines[54], lines[56]],
            ]

            eyes_triplets = [
                [lines[37], lines[36], lines[41]],
                [lines[38], lines[39], lines[40]],
                [lines[43], lines[42], lines[47]],
                [lines[44], lines[45], lines[46]],
            ]

            nose_triplets = [
                [lines[27], lines[31], lines[35]],
                [lines[30], lines[31], lines[35]],
                [lines[27], lines[35], lines[31]],
                [lines[30], lines[35], lines[31]],
            ]

            # angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in point_triplets]

            mouth_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in mouth_triplets]
            eyes_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in eyes_triplets]
            eye_angle_features = [0,0,0,0]
            nose_angle_features = [angle(triplet[0], triplet[1], triplet[2]) for triplet in nose_triplets]



            angle_features = []

            if "m" in self.includes:
                angle_features = angle_features + mouth_angle_features
            if "n" in self.includes:
                angle_features = angle_features + nose_angle_features
            if "e" in self.includes:
                angle_features = angle_features + eyes_angle_features


            angle_features = self.normalize_array(angle_features)

            return angle_features

        # get distance features
        dist_features = get_distance_features(lines)

        # get angle features
        angle_features = get_angle_features(self, lines)

        # combine the features
        # features = dist_features + angle_features
        # features = dist_features
        # print("features:", features)

        features = dist_features

        # if 'd' in self.includes:
        #     features = features + dist_features
        if 'a' in self.includes:
            features = features + angle_features

        return features

Model

In [ ]:
import os
import random
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_val_score

Predictor class


Model_Mouth

In [52]:
class Model_mouth():
    def __init__(self, test_size=0.2, low_number_of_images=2, high_number_of_images=1000, kernel_type="poly", proba = True, scale_feature = False, method = "svm", category=None):
        self.test_size = test_size
        self.low_number_of_images = low_number_of_images
        self.high_number_of_images = high_number_of_images
        self.kernel_type = kernel_type
        self.proba = proba

        self.feature_extractor = Features_Mouth()
        self.scale_feature = scale_feature
        self.method = method

        self.scaler = StandardScaler()


    def generate_train_test_split(self, category='neutral', includes='dabenm'):
        # print(landmark_path)

        #define features and labels
        train_features = []
        train_labels = []

        test_features = []
        test_labels = []
        #for each subfolder(labels) in the path
        cnt_valid_labels = 0
        for label in os.listdir(landmark_path):
            #get the number of images with only 1 person in the image
            cnt =len(os.listdir(landmark_path + label+ "/" +category))
            # for file_name in os.listdir(landmark_path + label+ "/" +category):
            #     file = open(landmark_path + label + "/" + category+"/"+ file_name, "r")
            #     lines = file.readlines()
            #     if len(lines) == 68:
            #         cnt += 1
            #     file.close()

            #if the number of images is between the low and high number of images
            if cnt >= self.low_number_of_images and cnt <= self.high_number_of_images:
                # print('here')
                cnt_valid_labels += 1

                #for each landmark file in the subfolder
                for file_name in os.listdir(landmark_path + label+ "/" +category):
                    # print(file_name)
                    file = open(landmark_path + label + "/" + category + "/" + file_name, "r")
                    lines = file.readlines()
                    if len(lines) == 68:
                        #get the features and labels for the image
                        # features = self.get_features(landmark_path + label + "/" + file_name)
                        features = self.feature_extractor.get_features(lines, includes)

                        #check if the image is in the test set or the train set
                        if random.random() < self.test_size:
                            test_features.append(features)
                            test_labels.append(label)
                        else:
                            train_features.append(features)
                            train_labels.append(label)

            #convert the features and labels to numpy arrays
            self.x_train = np.array(train_features, dtype=np.float64)
            self.x_test = np.array(test_features, dtype=np.float64)
            self.y_train = np.array(train_labels, dtype=object)
            self.y_test = np.array(test_labels, dtype=object)


        # print("Valid labels:", cnt_valid_labels)




    def train_model(self, category=None):

        if self.scale_feature:
          self.x_train = self.scaler.fit_transform(self.x_train)
          self.x_test = self.scaler.transform(self.x_test)

        if self.method == "svm":
          #train the SVM
          self.model = svm.SVC(kernel=self.kernel_type, probability=self.proba)
          self.model.fit(self.x_train, self.y_train)

        elif self.method == "rand_forest":
          #self.model= RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
          #cv = KFold(n_splits=10, random_state=1, shuffle=True)
          #self.model.fit(self.x_train, self.y_train)
          #cross_val_score(model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1)
          self.model = RandomForestClassifier()
          cv = KFold(n_splits=10, random_state=1, shuffle=True)
          self.model.fit(self.x_train, self.y_train)
          print(cross_val_score(self.model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1))

        #save the model
        if category is None:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model.pkl")
        else:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model_only_spon_" + category + ".pkl")

        # print("SVM model saved")

    def get_accuracy2(self, x_test=None, y_test=None):
        #get the accuracy of the model
        cm = None

        def second_max(lst):
            m1, m2 = max(lst), float('-inf')
            for x in lst:
                if x != m1:
                    m2 = max(m2, x)
            return m2

        if x_test is None or y_test is None:
          self.y_pred = self.model.predict(self.x_test);
          # cm = confusion matrix
          cm = confusion_matrix(self.y_test, self.y_pred, normalize='true')
        else:
          y_pred = self.model.predict(x_test)
          cm = confusion_matrix(y_test, y_pred, normalize='true')

        if cm is None:
            return 0.0, 0.0, 0.0

        accuracy_rate = sum(cm.diagonal()) / len(cm)
        not_sure_rate = sum(1 for row in cm if max(row) - second_max(row) < 0.1) / len(cm)
        inaccuracy_rate = 1 - accuracy_rate - not_sure_rate

        return accuracy_rate, not_sure_rate, inaccuracy_rate


    def get_accuracy(self, x_test=None, y_test=None):
        #get the accuracy of the model
        # predict probabilities of each instance

        if x_test is None or y_test is None:
            x_test = self.x_test
            y_test = self.y_test


        probabilities = self.model.predict_proba(x_test)

        undetermined = 0
        accurate = 0
        inaccurate = 0

        # loop through each instance and determine the prediction accuracy
        for i in range(len(x_test)):
            pred_class_idx = np.argmax(probabilities[i])
            pred_class = self.model.classes_[pred_class_idx]
            pred_prob = probabilities[i][pred_class_idx]
            second_highest_prob = sorted(probabilities[i])[-2]

            if (pred_prob - second_highest_prob) > 0.02:
                # print('a', pred_class, y_test[i])
                if pred_class == y_test[i]:
                    accurate += 1
                else:
                    inaccurate += 1
            else:
                undetermined += 1

        accuracy_rate = accurate / len(x_test)
        uncertainty = undetermined / len(x_test)
        inaccuracy_rate = inaccurate / len(x_test)

        return accuracy_rate, uncertainty, inaccuracy_rate

    def predict(self, image_path, includes='dabenm'):
        #predict the label of the image
        features = self.feature_extractor.get_features_from_path(image_path, includes)
        #convert the features to a numpy array
        features = np.array(features, dtype=np.float64)

        #reshape the features
        features = features.reshape((1, features.shape[0]))

        if self.scale_feature:
          features = self.scaler.transform(features)
        #predict the label with confidence
        prediction = self.model.predict(features)
        confidence = self.model.predict_proba(features)

        print(prediction)
        return prediction[0],confidence


    def get_testset(self):
        return self.x_test, self.y_test



Model_Nose

In [53]:
class Model_Nose():
    def __init__(self, test_size=0.2, low_number_of_images=2, high_number_of_images=1000, kernel_type="poly", proba = True, scale_feature = False, method = "svm", category=None):
        self.test_size = test_size
        self.low_number_of_images = low_number_of_images
        self.high_number_of_images = high_number_of_images
        self.kernel_type = kernel_type
        self.proba = proba

        self.feature_extractor = Features_Nose()
        self.scale_feature = scale_feature
        self.method = method

        self.scaler = StandardScaler()


    def generate_train_test_split(self, category='neutral', includes='dabenm'):
        # print(landmark_path)

        #define features and labels
        train_features = []
        train_labels = []

        test_features = []
        test_labels = []
        #for each subfolder(labels) in the path
        cnt_valid_labels = 0
        for label in os.listdir(landmark_path):
            #get the number of images with only 1 person in the image
            cnt =len(os.listdir(landmark_path + label+ "/" +category))
            # for file_name in os.listdir(landmark_path + label+ "/" +category):
            #     file = open(landmark_path + label + "/" + category+"/"+ file_name, "r")
            #     lines = file.readlines()
            #     if len(lines) == 68:
            #         cnt += 1
            #     file.close()

            #if the number of images is between the low and high number of images
            if cnt >= self.low_number_of_images and cnt <= self.high_number_of_images:
                # print('here')
                cnt_valid_labels += 1

                #for each landmark file in the subfolder
                for file_name in os.listdir(landmark_path + label+ "/" +category):
                    # print(file_name)
                    file = open(landmark_path + label + "/" + category + "/" + file_name, "r")
                    lines = file.readlines()
                    if len(lines) == 68:
                        #get the features and labels for the image
                        # features = self.get_features(landmark_path + label + "/" + file_name)
                        features = self.feature_extractor.get_features(lines, includes)

                        #check if the image is in the test set or the train set
                        if random.random() < self.test_size:
                            test_features.append(features)
                            test_labels.append(label)
                        else:
                            train_features.append(features)
                            train_labels.append(label)

            #convert the features and labels to numpy arrays
            self.x_train = np.array(train_features, dtype=np.float64)
            self.x_test = np.array(test_features, dtype=np.float64)
            self.y_train = np.array(train_labels, dtype=object)
            self.y_test = np.array(test_labels, dtype=object)


        # print("Valid labels:", cnt_valid_labels)




    def train_model(self, category=None):

        if self.scale_feature:
          self.x_train = self.scaler.fit_transform(self.x_train)
          self.x_test = self.scaler.transform(self.x_test)

        if self.method == "svm":
          #train the SVM
          self.model = svm.SVC(kernel=self.kernel_type, probability=self.proba)
          self.model.fit(self.x_train, self.y_train)

        elif self.method == "rand_forest":
          #self.model= RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
          #cv = KFold(n_splits=10, random_state=1, shuffle=True)
          #self.model.fit(self.x_train, self.y_train)
          #cross_val_score(model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1)
          self.model = RandomForestClassifier()
          cv = KFold(n_splits=10, random_state=1, shuffle=True)
          self.model.fit(self.x_train, self.y_train)
          print(cross_val_score(self.model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1))

        #save the model
        if category is None:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model.pkl")
        else:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model_only_spon_" + category + ".pkl")

        # print("SVM model saved")

    def get_accuracy2(self, x_test=None, y_test=None):
        #get the accuracy of the model
        cm = None

        def second_max(lst):
            m1, m2 = max(lst), float('-inf')
            for x in lst:
                if x != m1:
                    m2 = max(m2, x)
            return m2

        if x_test is None or y_test is None:
          self.y_pred = self.model.predict(self.x_test);
          # cm = confusion matrix
          cm = confusion_matrix(self.y_test, self.y_pred, normalize='true')
        else:
          y_pred = self.model.predict(x_test)
          cm = confusion_matrix(y_test, y_pred, normalize='true')

        if cm is None:
            return 0.0, 0.0, 0.0

        accuracy_rate = sum(cm.diagonal()) / len(cm)
        not_sure_rate = sum(1 for row in cm if max(row) - second_max(row) < 0.1) / len(cm)
        inaccuracy_rate = 1 - accuracy_rate - not_sure_rate

        return accuracy_rate, not_sure_rate, inaccuracy_rate


    def get_accuracy(self, x_test=None, y_test=None):
        #get the accuracy of the model
        # predict probabilities of each instance

        if x_test is None or y_test is None:
            x_test = self.x_test
            y_test = self.y_test


        probabilities = self.model.predict_proba(x_test)

        undetermined = 0
        accurate = 0
        inaccurate = 0

        # loop through each instance and determine the prediction accuracy
        for i in range(len(x_test)):
            pred_class_idx = np.argmax(probabilities[i])
            pred_class = self.model.classes_[pred_class_idx]
            pred_prob = probabilities[i][pred_class_idx]
            second_highest_prob = sorted(probabilities[i])[-2]

            if (pred_prob - second_highest_prob) > 0.02:
                # print('a', pred_class, y_test[i])
                if pred_class == y_test[i]:
                    accurate += 1
                else:
                    inaccurate += 1
            else:
                undetermined += 1

        accuracy_rate = accurate / len(x_test)
        uncertainty = undetermined / len(x_test)
        inaccuracy_rate = inaccurate / len(x_test)

        return accuracy_rate, uncertainty, inaccuracy_rate

    def predict(self, image_path, includes='dabenm'):
        #predict the label of the image
        features = self.feature_extractor.get_features_from_path(image_path, includes)
        #convert the features to a numpy array
        features = np.array(features, dtype=np.float64)

        #reshape the features
        features = features.reshape((1, features.shape[0]))

        if self.scale_feature:
          features = self.scaler.transform(features)
        #predict the label with confidence
        prediction = self.model.predict(features)
        confidence = self.model.predict_proba(features)

        print(prediction)
        return prediction[0],confidence


    def get_testset(self):
        return self.x_test, self.y_test


Model_Eye

In [54]:
class Model_eye():
    def __init__(self, test_size=0.2, low_number_of_images=2, high_number_of_images=1000, kernel_type="poly", proba = True, scale_feature = False, method = "svm", category=None):
        self.test_size = test_size
        self.low_number_of_images = low_number_of_images
        self.high_number_of_images = high_number_of_images
        self.kernel_type = kernel_type
        self.proba = proba

        self.feature_extractor = Features_Eye()
        self.scale_feature = scale_feature
        self.method = method

        self.scaler = StandardScaler()


    def generate_train_test_split(self, category='neutral', includes='dabenm'):
        # print(landmark_path)

        #define features and labels
        train_features = []
        train_labels = []

        test_features = []
        test_labels = []
        #for each subfolder(labels) in the path
        cnt_valid_labels = 0
        for label in os.listdir(landmark_path):
            #get the number of images with only 1 person in the image
            cnt =len(os.listdir(landmark_path + label+ "/" +category))
            # for file_name in os.listdir(landmark_path + label+ "/" +category):
            #     file = open(landmark_path + label + "/" + category+"/"+ file_name, "r")
            #     lines = file.readlines()
            #     if len(lines) == 68:
            #         cnt += 1
            #     file.close()

            #if the number of images is between the low and high number of images
            if cnt >= self.low_number_of_images and cnt <= self.high_number_of_images:
                # print('here')
                cnt_valid_labels += 1

                #for each landmark file in the subfolder
                for file_name in os.listdir(landmark_path + label+ "/" +category):
                    # print(file_name)
                    file = open(landmark_path + label + "/" + category + "/" + file_name, "r")
                    lines = file.readlines()
                    if len(lines) == 68:
                        #get the features and labels for the image
                        # features = self.get_features(landmark_path + label + "/" + file_name)
                        features = self.feature_extractor.get_features(lines, includes)

                        #check if the image is in the test set or the train set
                        if random.random() < self.test_size:
                            test_features.append(features)
                            test_labels.append(label)
                        else:
                            train_features.append(features)
                            train_labels.append(label)

            #convert the features and labels to numpy arrays
            self.x_train = np.array(train_features, dtype=np.float64)
            self.x_test = np.array(test_features, dtype=np.float64)
            self.y_train = np.array(train_labels, dtype=object)
            self.y_test = np.array(test_labels, dtype=object)


        # print("Valid labels:", cnt_valid_labels)




    def train_model(self, category=None):

        if self.scale_feature:
          self.x_train = self.scaler.fit_transform(self.x_train)
          self.x_test = self.scaler.transform(self.x_test)

        if self.method == "svm":
          #train the SVM
          self.model = svm.SVC(kernel=self.kernel_type, probability=self.proba)
          self.model.fit(self.x_train, self.y_train)

        elif self.method == "rand_forest":
          #self.model= RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
          #cv = KFold(n_splits=10, random_state=1, shuffle=True)
          #self.model.fit(self.x_train, self.y_train)
          #cross_val_score(model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1)
          self.model = RandomForestClassifier()
          cv = KFold(n_splits=10, random_state=1, shuffle=True)
          self.model.fit(self.x_train, self.y_train)
          print(cross_val_score(self.model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1))

        #save the model
        if category is None:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model.pkl")
        else:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model_only_spon_" + category + ".pkl")

        # print("SVM model saved")

    def get_accuracy2(self, x_test=None, y_test=None):
        #get the accuracy of the model
        cm = None

        def second_max(lst):
            m1, m2 = max(lst), float('-inf')
            for x in lst:
                if x != m1:
                    m2 = max(m2, x)
            return m2

        if x_test is None or y_test is None:
          self.y_pred = self.model.predict(self.x_test);
          # cm = confusion matrix
          cm = confusion_matrix(self.y_test, self.y_pred, normalize='true')
        else:
          y_pred = self.model.predict(x_test)
          cm = confusion_matrix(y_test, y_pred, normalize='true')

        if cm is None:
            return 0.0, 0.0, 0.0

        accuracy_rate = sum(cm.diagonal()) / len(cm)
        not_sure_rate = sum(1 for row in cm if max(row) - second_max(row) < 0.1) / len(cm)
        inaccuracy_rate = 1 - accuracy_rate - not_sure_rate

        return accuracy_rate, not_sure_rate, inaccuracy_rate


    def get_accuracy(self, x_test=None, y_test=None):
        #get the accuracy of the model
        # predict probabilities of each instance

        if x_test is None or y_test is None:
            x_test = self.x_test
            y_test = self.y_test


        probabilities = self.model.predict_proba(x_test)

        undetermined = 0
        accurate = 0
        inaccurate = 0

        # loop through each instance and determine the prediction accuracy
        for i in range(len(x_test)):
            pred_class_idx = np.argmax(probabilities[i])
            pred_class = self.model.classes_[pred_class_idx]
            pred_prob = probabilities[i][pred_class_idx]
            second_highest_prob = sorted(probabilities[i])[-2]

            if (pred_prob - second_highest_prob) > 0.02:
                # print('a', pred_class, y_test[i])
                if pred_class == y_test[i]:
                    accurate += 1
                else:
                    inaccurate += 1
            else:
                undetermined += 1

        accuracy_rate = accurate / len(x_test)
        uncertainty = undetermined / len(x_test)
        inaccuracy_rate = inaccurate / len(x_test)

        return accuracy_rate, uncertainty, inaccuracy_rate

    def predict(self, image_path, includes='dabenm'):
        #predict the label of the image
        features = self.feature_extractor.get_features_from_path(image_path, includes)
        #convert the features to a numpy array
        features = np.array(features, dtype=np.float64)

        #reshape the features
        features = features.reshape((1, features.shape[0]))

        if self.scale_feature:
          features = self.scaler.transform(features)
        #predict the label with confidence
        prediction = self.model.predict(features)
        confidence = self.model.predict_proba(features)

        print(prediction)
        return prediction[0],confidence


    def get_testset(self):
        return self.x_test, self.y_test


Model Mouth-Eye

In [55]:
class Model_mouth_eye():
    def __init__(self, test_size=0.2, low_number_of_images=2, high_number_of_images=1000, kernel_type="poly", proba = True, scale_feature = False, method = "svm", category=None):
        self.test_size = test_size
        self.low_number_of_images = low_number_of_images
        self.high_number_of_images = high_number_of_images
        self.kernel_type = kernel_type
        self.proba = proba

        self.feature_extractor = Features_mouth_eye()
        self.scale_feature = scale_feature
        self.method = method

        self.scaler = StandardScaler()


    def generate_train_test_split(self, category='neutral', includes='dabenm'):
        # print(landmark_path)

        #define features and labels
        train_features = []
        train_labels = []

        test_features = []
        test_labels = []
        #for each subfolder(labels) in the path
        cnt_valid_labels = 0
        for label in os.listdir(landmark_path):
            #get the number of images with only 1 person in the image
            cnt =len(os.listdir(landmark_path + label+ "/" +category))
            # for file_name in os.listdir(landmark_path + label+ "/" +category):
            #     file = open(landmark_path + label + "/" + category+"/"+ file_name, "r")
            #     lines = file.readlines()
            #     if len(lines) == 68:
            #         cnt += 1
            #     file.close()

            #if the number of images is between the low and high number of images
            if cnt >= self.low_number_of_images and cnt <= self.high_number_of_images:
                # print('here')
                cnt_valid_labels += 1

                #for each landmark file in the subfolder
                for file_name in os.listdir(landmark_path + label+ "/" +category):
                    # print(file_name)
                    file = open(landmark_path + label + "/" + category + "/" + file_name, "r")
                    lines = file.readlines()
                    if len(lines) == 68:
                        #get the features and labels for the image
                        # features = self.get_features(landmark_path + label + "/" + file_name)
                        features = self.feature_extractor.get_features(lines, includes)

                        #check if the image is in the test set or the train set
                        if random.random() < self.test_size:
                            test_features.append(features)
                            test_labels.append(label)
                        else:
                            train_features.append(features)
                            train_labels.append(label)

            #convert the features and labels to numpy arrays
            self.x_train = np.array(train_features, dtype=np.float64)
            self.x_test = np.array(test_features, dtype=np.float64)
            self.y_train = np.array(train_labels, dtype=object)
            self.y_test = np.array(test_labels, dtype=object)


        # print("Valid labels:", cnt_valid_labels)




    def train_model(self, category=None):

        if self.scale_feature:
          self.x_train = self.scaler.fit_transform(self.x_train)
          self.x_test = self.scaler.transform(self.x_test)

        if self.method == "svm":
          #train the SVM
          self.model = svm.SVC(kernel=self.kernel_type, probability=self.proba)
          self.model.fit(self.x_train, self.y_train)

        elif self.method == "rand_forest":
          #self.model= RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
          #cv = KFold(n_splits=10, random_state=1, shuffle=True)
          #self.model.fit(self.x_train, self.y_train)
          #cross_val_score(model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1)
          self.model = RandomForestClassifier()
          cv = KFold(n_splits=10, random_state=1, shuffle=True)
          self.model.fit(self.x_train, self.y_train)
          print(cross_val_score(self.model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1))

        #save the model
        if category is None:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model.pkl")
        else:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model_only_spon_" + category + ".pkl")

        # print("SVM model saved")

    def get_accuracy2(self, x_test=None, y_test=None):
        #get the accuracy of the model
        cm = None

        def second_max(lst):
            m1, m2 = max(lst), float('-inf')
            for x in lst:
                if x != m1:
                    m2 = max(m2, x)
            return m2

        if x_test is None or y_test is None:
          self.y_pred = self.model.predict(self.x_test);
          # cm = confusion matrix
          cm = confusion_matrix(self.y_test, self.y_pred, normalize='true')
        else:
          y_pred = self.model.predict(x_test)
          cm = confusion_matrix(y_test, y_pred, normalize='true')

        if cm is None:
            return 0.0, 0.0, 0.0

        accuracy_rate = sum(cm.diagonal()) / len(cm)
        not_sure_rate = sum(1 for row in cm if max(row) - second_max(row) < 0.1) / len(cm)
        inaccuracy_rate = 1 - accuracy_rate - not_sure_rate

        return accuracy_rate, not_sure_rate, inaccuracy_rate


    def get_accuracy(self, x_test=None, y_test=None):
        #get the accuracy of the model
        # predict probabilities of each instance

        if x_test is None or y_test is None:
            x_test = self.x_test
            y_test = self.y_test


        probabilities = self.model.predict_proba(x_test)

        undetermined = 0
        accurate = 0
        inaccurate = 0

        # loop through each instance and determine the prediction accuracy
        for i in range(len(x_test)):
            pred_class_idx = np.argmax(probabilities[i])
            pred_class = self.model.classes_[pred_class_idx]
            pred_prob = probabilities[i][pred_class_idx]
            second_highest_prob = sorted(probabilities[i])[-2]

            if (pred_prob - second_highest_prob) > 0.02:
                # print('a', pred_class, y_test[i])
                if pred_class == y_test[i]:
                    accurate += 1
                else:
                    inaccurate += 1
            else:
                undetermined += 1

        accuracy_rate = accurate / len(x_test)
        uncertainty = undetermined / len(x_test)
        inaccuracy_rate = inaccurate / len(x_test)

        return accuracy_rate, uncertainty, inaccuracy_rate

    def predict(self, image_path, includes='dabenm'):
        #predict the label of the image
        features = self.feature_extractor.get_features_from_path(image_path, includes)
        #convert the features to a numpy array
        features = np.array(features, dtype=np.float64)

        #reshape the features
        features = features.reshape((1, features.shape[0]))

        if self.scale_feature:
          features = self.scaler.transform(features)
        #predict the label with confidence
        prediction = self.model.predict(features)
        confidence = self.model.predict_proba(features)

        print(prediction)
        return prediction[0],confidence


    def get_testset(self):
        return self.x_test, self.y_test

Model Eye-Nose

In [56]:
class Model_eye_nose():
    def __init__(self, test_size=0.2, low_number_of_images=2, high_number_of_images=1000, kernel_type="poly", proba = True, scale_feature = False, method = "svm", category=None):
        self.test_size = test_size
        self.low_number_of_images = low_number_of_images
        self.high_number_of_images = high_number_of_images
        self.kernel_type = kernel_type
        self.proba = proba

        self.feature_extractor = Features_eye_nose()
        self.scale_feature = scale_feature
        self.method = method

        self.scaler = StandardScaler()


    def generate_train_test_split(self, category='neutral', includes='dabenm'):
        # print(landmark_path)

        #define features and labels
        train_features = []
        train_labels = []

        test_features = []
        test_labels = []
        #for each subfolder(labels) in the path
        cnt_valid_labels = 0
        for label in os.listdir(landmark_path):
            #get the number of images with only 1 person in the image
            cnt =len(os.listdir(landmark_path + label+ "/" +category))
            # for file_name in os.listdir(landmark_path + label+ "/" +category):
            #     file = open(landmark_path + label + "/" + category+"/"+ file_name, "r")
            #     lines = file.readlines()
            #     if len(lines) == 68:
            #         cnt += 1
            #     file.close()

            #if the number of images is between the low and high number of images
            if cnt >= self.low_number_of_images and cnt <= self.high_number_of_images:
                # print('here')
                cnt_valid_labels += 1

                #for each landmark file in the subfolder
                for file_name in os.listdir(landmark_path + label+ "/" +category):
                    # print(file_name)
                    file = open(landmark_path + label + "/" + category + "/" + file_name, "r")
                    lines = file.readlines()
                    if len(lines) == 68:
                        #get the features and labels for the image
                        # features = self.get_features(landmark_path + label + "/" + file_name)
                        features = self.feature_extractor.get_features(lines, includes)

                        #check if the image is in the test set or the train set
                        if random.random() < self.test_size:
                            test_features.append(features)
                            test_labels.append(label)
                        else:
                            train_features.append(features)
                            train_labels.append(label)

            #convert the features and labels to numpy arrays
            self.x_train = np.array(train_features, dtype=np.float64)
            self.x_test = np.array(test_features, dtype=np.float64)
            self.y_train = np.array(train_labels, dtype=object)
            self.y_test = np.array(test_labels, dtype=object)


        # print("Valid labels:", cnt_valid_labels)




    def train_model(self, category=None):

        if self.scale_feature:
          self.x_train = self.scaler.fit_transform(self.x_train)
          self.x_test = self.scaler.transform(self.x_test)

        if self.method == "svm":
          #train the SVM
          self.model = svm.SVC(kernel=self.kernel_type, probability=self.proba)
          self.model.fit(self.x_train, self.y_train)

        elif self.method == "rand_forest":
          #self.model= RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
          #cv = KFold(n_splits=10, random_state=1, shuffle=True)
          #self.model.fit(self.x_train, self.y_train)
          #cross_val_score(model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1)
          self.model = RandomForestClassifier()
          cv = KFold(n_splits=10, random_state=1, shuffle=True)
          self.model.fit(self.x_train, self.y_train)
          print(cross_val_score(self.model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1))

        #save the model
        if category is None:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model.pkl")
        else:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model_only_spon_" + category + ".pkl")

        # print("SVM model saved")

    def get_accuracy2(self, x_test=None, y_test=None):
        #get the accuracy of the model
        cm = None

        def second_max(lst):
            m1, m2 = max(lst), float('-inf')
            for x in lst:
                if x != m1:
                    m2 = max(m2, x)
            return m2

        if x_test is None or y_test is None:
          self.y_pred = self.model.predict(self.x_test);
          # cm = confusion matrix
          cm = confusion_matrix(self.y_test, self.y_pred, normalize='true')
        else:
          y_pred = self.model.predict(x_test)
          cm = confusion_matrix(y_test, y_pred, normalize='true')

        if cm is None:
            return 0.0, 0.0, 0.0

        accuracy_rate = sum(cm.diagonal()) / len(cm)
        not_sure_rate = sum(1 for row in cm if max(row) - second_max(row) < 0.1) / len(cm)
        inaccuracy_rate = 1 - accuracy_rate - not_sure_rate

        return accuracy_rate, not_sure_rate, inaccuracy_rate


    def get_accuracy(self, x_test=None, y_test=None):
        #get the accuracy of the model
        # predict probabilities of each instance

        if x_test is None or y_test is None:
            x_test = self.x_test
            y_test = self.y_test


        probabilities = self.model.predict_proba(x_test)

        undetermined = 0
        accurate = 0
        inaccurate = 0

        # loop through each instance and determine the prediction accuracy
        for i in range(len(x_test)):
            pred_class_idx = np.argmax(probabilities[i])
            pred_class = self.model.classes_[pred_class_idx]
            pred_prob = probabilities[i][pred_class_idx]
            second_highest_prob = sorted(probabilities[i])[-2]

            if (pred_prob - second_highest_prob) > 0.02:
                # print('a', pred_class, y_test[i])
                if pred_class == y_test[i]:
                    accurate += 1
                else:
                    inaccurate += 1
            else:
                undetermined += 1

        accuracy_rate = accurate / len(x_test)
        uncertainty = undetermined / len(x_test)
        inaccuracy_rate = inaccurate / len(x_test)

        return accuracy_rate, uncertainty, inaccuracy_rate

    def predict(self, image_path, includes='dabenm'):
        #predict the label of the image
        features = self.feature_extractor.get_features_from_path(image_path, includes)
        #convert the features to a numpy array
        features = np.array(features, dtype=np.float64)

        #reshape the features
        features = features.reshape((1, features.shape[0]))

        if self.scale_feature:
          features = self.scaler.transform(features)
        #predict the label with confidence
        prediction = self.model.predict(features)
        confidence = self.model.predict_proba(features)

        print(prediction)
        return prediction[0],confidence


    def get_testset(self):
        return self.x_test, self.y_test

Model Mouth-Nose

In [57]:
class Model_mouth_nose():
    def __init__(self, test_size=0.2, low_number_of_images=2, high_number_of_images=1000, kernel_type="poly", proba = True, scale_feature = False, method = "svm", category=None):
        self.test_size = test_size
        self.low_number_of_images = low_number_of_images
        self.high_number_of_images = high_number_of_images
        self.kernel_type = kernel_type
        self.proba = proba

        self.feature_extractor = Features_mouth_nose()
        self.scale_feature = scale_feature
        self.method = method

        self.scaler = StandardScaler()


    def generate_train_test_split(self, category='neutral', includes='dabenm'):
        # print(landmark_path)

        #define features and labels
        train_features = []
        train_labels = []

        test_features = []
        test_labels = []
        #for each subfolder(labels) in the path
        cnt_valid_labels = 0
        for label in os.listdir(landmark_path):
            #get the number of images with only 1 person in the image
            cnt =len(os.listdir(landmark_path + label+ "/" +category))
            # for file_name in os.listdir(landmark_path + label+ "/" +category):
            #     file = open(landmark_path + label + "/" + category+"/"+ file_name, "r")
            #     lines = file.readlines()
            #     if len(lines) == 68:
            #         cnt += 1
            #     file.close()

            #if the number of images is between the low and high number of images
            if cnt >= self.low_number_of_images and cnt <= self.high_number_of_images:
                # print('here')
                cnt_valid_labels += 1

                #for each landmark file in the subfolder
                for file_name in os.listdir(landmark_path + label+ "/" +category):
                    # print(file_name)
                    file = open(landmark_path + label + "/" + category + "/" + file_name, "r")
                    lines = file.readlines()
                    if len(lines) == 68:
                        #get the features and labels for the image
                        # features = self.get_features(landmark_path + label + "/" + file_name)
                        features = self.feature_extractor.get_features(lines, includes)

                        #check if the image is in the test set or the train set
                        if random.random() < self.test_size:
                            test_features.append(features)
                            test_labels.append(label)
                        else:
                            train_features.append(features)
                            train_labels.append(label)

            #convert the features and labels to numpy arrays
            self.x_train = np.array(train_features, dtype=np.float64)
            self.x_test = np.array(test_features, dtype=np.float64)
            self.y_train = np.array(train_labels, dtype=object)
            self.y_test = np.array(test_labels, dtype=object)


        # print("Valid labels:", cnt_valid_labels)




    def train_model(self, category=None):

        if self.scale_feature:
          self.x_train = self.scaler.fit_transform(self.x_train)
          self.x_test = self.scaler.transform(self.x_test)

        if self.method == "svm":
          #train the SVM
          self.model = svm.SVC(kernel=self.kernel_type, probability=self.proba)
          self.model.fit(self.x_train, self.y_train)

        elif self.method == "rand_forest":
          #self.model= RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 42)
          #cv = KFold(n_splits=10, random_state=1, shuffle=True)
          #self.model.fit(self.x_train, self.y_train)
          #cross_val_score(model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1)
          self.model = RandomForestClassifier()
          cv = KFold(n_splits=10, random_state=1, shuffle=True)
          self.model.fit(self.x_train, self.y_train)
          print(cross_val_score(self.model, self.x_train, self.y_train, scoring='accuracy', cv=cv, n_jobs=-1))

        #save the model
        if category is None:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model.pkl")
        else:
          joblib.dump(self.model, "/content/drive/MyDrive/Msthesis/model/svm_model_only_spon_" + category + ".pkl")

        # print("SVM model saved")

    def get_accuracy2(self, x_test=None, y_test=None):
        #get the accuracy of the model
        cm = None

        def second_max(lst):
            m1, m2 = max(lst), float('-inf')
            for x in lst:
                if x != m1:
                    m2 = max(m2, x)
            return m2

        if x_test is None or y_test is None:
          self.y_pred = self.model.predict(self.x_test);
          # cm = confusion matrix
          cm = confusion_matrix(self.y_test, self.y_pred, normalize='true')
        else:
          y_pred = self.model.predict(x_test)
          cm = confusion_matrix(y_test, y_pred, normalize='true')

        if cm is None:
            return 0.0, 0.0, 0.0

        accuracy_rate = sum(cm.diagonal()) / len(cm)
        not_sure_rate = sum(1 for row in cm if max(row) - second_max(row) < 0.1) / len(cm)
        inaccuracy_rate = 1 - accuracy_rate - not_sure_rate

        return accuracy_rate, not_sure_rate, inaccuracy_rate


    def get_accuracy(self, x_test=None, y_test=None):
        #get the accuracy of the model
        # predict probabilities of each instance

        if x_test is None or y_test is None:
            x_test = self.x_test
            y_test = self.y_test


        probabilities = self.model.predict_proba(x_test)

        undetermined = 0
        accurate = 0
        inaccurate = 0

        # loop through each instance and determine the prediction accuracy
        for i in range(len(x_test)):
            pred_class_idx = np.argmax(probabilities[i])
            pred_class = self.model.classes_[pred_class_idx]
            pred_prob = probabilities[i][pred_class_idx]
            second_highest_prob = sorted(probabilities[i])[-2]

            if (pred_prob - second_highest_prob) > 0.02:
                # print('a', pred_class, y_test[i])
                if pred_class == y_test[i]:
                    accurate += 1
                else:
                    inaccurate += 1
            else:
                undetermined += 1

        accuracy_rate = accurate / len(x_test)
        uncertainty = undetermined / len(x_test)
        inaccuracy_rate = inaccurate / len(x_test)

        return accuracy_rate, uncertainty, inaccuracy_rate

    def predict(self, image_path, includes='dabenm'):
        #predict the label of the image
        features = self.feature_extractor.get_features_from_path(image_path, includes)
        #convert the features to a numpy array
        features = np.array(features, dtype=np.float64)

        #reshape the features
        features = features.reshape((1, features.shape[0]))

        if self.scale_feature:
          features = self.scaler.transform(features)
        #predict the label with confidence
        prediction = self.model.predict(features)
        confidence = self.model.predict_proba(features)

        print(prediction)
        return prediction[0],confidence


    def get_testset(self):
        return self.x_test, self.y_test

Test with each combination

In [58]:
includes = 'dabenm'

In [59]:
# Mouth features Model
sv_m = Model_mouth(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
sv_m.generate_train_test_split(category='neutral', includes=includes)
sv_m.train_model(category='neutral')

[0.71929825 0.65789474 0.71052632 0.71929825 0.69298246 0.69298246
 0.76315789 0.69298246 0.73684211 0.77192982]


In [60]:
# Nose features Model
sv_n = Model_Nose(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
sv_n.generate_train_test_split(category='neutral', includes=includes)
sv_n.train_model(category='neutral')

[0.66949153 0.71186441 0.68644068 0.7008547  0.73504274 0.72649573
 0.73504274 0.66666667 0.76068376 0.73504274]


In [61]:
# Eye features Model
sv_e = Model_eye(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
sv_e.generate_train_test_split(category='neutral', includes=includes)
sv_e.train_model(category='neutral')

[0.69230769 0.71794872 0.75       0.64655172 0.72413793 0.80172414
 0.6637931  0.73275862 0.71551724 0.71551724]


In [62]:
# Mouth_eye features Model
sv_me = Model_mouth_eye(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
sv_me.generate_train_test_split(category='neutral', includes=includes)
sv_me.train_model(category='neutral')

[0.74336283 0.73451327 0.67256637 0.68141593 0.78761062 0.74336283
 0.68141593 0.77678571 0.66964286 0.66964286]


In [63]:
# Eye-Nose features Model
sv_en = Model_eye_nose(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
sv_en.generate_train_test_split(category='neutral', includes=includes)
sv_en.train_model(category='neutral')

[0.72807018 0.77192982 0.68421053 0.66371681 0.7079646  0.62831858
 0.73451327 0.77876106 0.73451327 0.6460177 ]


In [64]:
# Mouth-Nose features Model
sv_mn = Model_mouth_nose(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
sv_mn.generate_train_test_split(category='neutral', includes=includes)
sv_mn.train_model(category='neutral')

[0.64655172 0.70689655 0.64655172 0.71304348 0.73913043 0.70434783
 0.72173913 0.77391304 0.7826087  0.69565217]


In [65]:
print('neutral - neutral')
# neutral Face-neutral Mouth
accuracy_rate, not_sure_rate, inaccuracy_rate = sv_m.get_accuracy()
print('neutral Face-neutral Mouth')
print("\taccuracy_rate: {:.2f}%".format(accuracy_rate * 100))
print("\tnot_sure_rate: {:.2f}%".format(not_sure_rate * 100))
print("\tinaccuracy_rate: {:.2f}%".format(inaccuracy_rate * 100))

# neutral Face-neutral Nose

accuracy_rate, not_sure_rate, inaccuracy_rate = sv_n.get_accuracy()
print('neutral Face-neutral Nose')
print("\taccuracy_rate: {:.2f}%".format(accuracy_rate * 100))
print("\tnot_sure_rate: {:.2f}%".format(not_sure_rate * 100))
print("\tinaccuracy_rate: {:.2f}%".format(inaccuracy_rate * 100))

# neutral Face-neutral Eye
accuracy_rate, not_sure_rate, inaccuracy_rate = sv_e.get_accuracy()
print('neutral Face-neutral Eye')
print("\taccuracy_rate: {:.2f}%".format(accuracy_rate * 100))
print("\tnot_sure_rate: {:.2f}%".format(not_sure_rate * 100))
print("\tinaccuracy_rate: {:.2f}%".format(inaccuracy_rate * 100))

# neutral Face-neutral Mouth_Eye

accuracy_rate, not_sure_rate, inaccuracy_rate = sv_me.get_accuracy()
print('neutral Face-neutral Mouth_Eye')
print("\taccuracy_rate: {:.2f}%".format(accuracy_rate * 100))
print("\tnot_sure_rate: {:.2f}%".format(not_sure_rate * 100))
print("\tinaccuracy_rate: {:.2f}%".format(inaccuracy_rate * 100))

# neutral Face-neutral Eye_Nose

accuracy_rate, not_sure_rate, inaccuracy_rate = sv_en.get_accuracy()
print('neutral Face-neutral Eye_Nose')
print("\taccuracy_rate: {:.2f}%".format(accuracy_rate * 100))
print("\tnot_sure_rate: {:.2f}%".format(not_sure_rate * 100))
print("\tinaccuracy_rate: {:.2f}%".format(inaccuracy_rate * 100))

# neutral Face-neutral Mouth_Nose

accuracy_rate, not_sure_rate, inaccuracy_rate = sv_mn.get_accuracy()
print('neutral Face-neutral Mouth_Nose')
print("\taccuracy_rate: {:.2f}%".format(accuracy_rate * 100))
print("\tnot_sure_rate: {:.2f}%".format(not_sure_rate * 100))
print("\tinaccuracy_rate: {:.2f}%".format(inaccuracy_rate * 100))


neutral - neutral
neutral Face-neutral Mouth
	accuracy_rate: 68.01%
	not_sure_rate: 16.16%
	inaccuracy_rate: 15.82%
neutral Face-neutral Nose
	accuracy_rate: 73.11%
	not_sure_rate: 12.12%
	inaccuracy_rate: 14.77%
neutral Face-neutral Eye
	accuracy_rate: 64.36%
	not_sure_rate: 18.91%
	inaccuracy_rate: 16.73%
neutral Face-neutral Mouth_Eye
	accuracy_rate: 69.68%
	not_sure_rate: 15.48%
	inaccuracy_rate: 14.84%
neutral Face-neutral Eye_Nose
	accuracy_rate: 67.11%
	not_sure_rate: 18.42%
	inaccuracy_rate: 14.47%
neutral Face-neutral Mouth_Nose
	accuracy_rate: 71.13%
	not_sure_rate: 17.25%
	inaccuracy_rate: 11.62%


Test Multiple times and take average

In [67]:
includes = 'ns'
MAX_ITER = 1
METHOD = 'svm'

In [68]:
# Mouth Model
def get_mouth_model(includes='ns'):
    sv_m = Model_mouth(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
    sv_m.generate_train_test_split(category='neutral', includes=includes)
    sv_m.train_model(category='neutral')
    return sv_m

In [69]:
# Nose Model
def get_nose_model(includes='ns'):
    sv_n = Model_Nose(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
    sv_n.generate_train_test_split(category='neutral', includes=includes)
    sv_n.train_model(category='neutral')
    return sv_n

In [70]:
# Eye Model
def get_eye_model(includes='ns'):
    sv_e = Model_eye(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
    sv_e.generate_train_test_split(category='neutral', includes=includes)
    sv_e.train_model(category='neutral')
    return sv_e

In [71]:
# Mouth-Eye Model
def get_mouth_eye_model(includes='ns'):
    sv_me = Model_mouth_eye(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
    sv_me.generate_train_test_split(category='neutral', includes=includes)
    sv_me.train_model(category='neutral')
    return sv_e

In [72]:
# # Eye-Nose features Model
def get_eye_nose_model(includes='ns'):
    sv_en = Model_eye_nose(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
    sv_en.generate_train_test_split(category='neutral', includes=includes)
    sv_en.train_model(category='neutral')
    return sv_en

In [73]:
#  # Mouth-Nose features Model
def get_mouth_nose_model(includes='ns'):
    sv_mn = Model_mouth_nose(test_size=0.2, low_number_of_images=4, kernel_type = "poly", scale_feature= True, method="rand_forest", category='neutral')
    sv_mn.generate_train_test_split(category='neutral', includes=includes)
    sv_mn.train_model(category='neutral')
    return sv_mn

In [74]:
rates = np.zeros(20)
print('\r' + '-'*int(MAX_ITER*100/MAX_ITER), end=f' {0}%')
for i in range(MAX_ITER):
    # print(i, end=' ')
    mouth_model = get_mouth_model(includes=includes)
    nose_model = get_nose_model(includes=includes)
    eye_model = get_eye_model(includes=includes)
    mouth_eye_model = get_mouth_eye_model(includes=includes)
    eye_nose_model = get_eye_nose_model(includes=includes)
    mouth_nose_model = get_mouth_nose_model(includes=includes)

    print('\r' + '*'*int((i+0.5)*100/MAX_ITER) + '-'*int((MAX_ITER-i-0.5)*100/MAX_ITER), end=f' {(i+0.5)/MAX_ITER * 100}%')
    # neutral-Mouth
    accuracy_rate, not_sure_rate, inaccuracy_rate = sv_m.get_accuracy()
    rates[0] += accuracy_rate
    rates[1] += not_sure_rate
    rates[2] += inaccuracy_rate

    # neutral-Nose
    accuracy_rate, not_sure_rate, inaccuracy_rate = sv_n.get_accuracy()
    rates[3] += accuracy_rate
    rates[4] += not_sure_rate
    rates[5] += inaccuracy_rate

    # neutral-Eye
    accuracy_rate, not_sure_rate, inaccuracy_rate = sv_e.get_accuracy()
    rates[6] += accuracy_rate
    rates[7] += not_sure_rate
    rates[8] += inaccuracy_rate

    # neutral-Mouth_Eye
    accuracy_rate, not_sure_rate, inaccuracy_rate = sv_me.get_accuracy()
    rates[9] += accuracy_rate
    rates[10] += not_sure_rate
    rates[11] += inaccuracy_rate

    # neutral-Eye_Nose
    accuracy_rate, not_sure_rate, inaccuracy_rate = sv_en.get_accuracy()
    rates[12] += accuracy_rate
    rates[13] += not_sure_rate
    rates[14] += inaccuracy_rate

    # neutral-Mouth_Nose
    accuracy_rate, not_sure_rate, inaccuracy_rate = sv_mn.get_accuracy()
    rates[15] += accuracy_rate
    rates[16] += not_sure_rate
    rates[17] += inaccuracy_rate


    print('\r' + '*'*int((i+1)*100/MAX_ITER) + '-'*int((MAX_ITER-i-1)*100/MAX_ITER), end=f' {(i+1)/MAX_ITER * 100}%')

rates = rates / MAX_ITER

---------------------------------------------------------------------------------------------------- 0%[0.37931034 0.48275862 0.4137931  0.46551724 0.45689655 0.40517241
 0.44827586 0.49565217 0.54782609 0.49565217]
[0.4173913  0.4173913  0.3826087  0.46956522 0.40350877 0.54385965
 0.44736842 0.37719298 0.47368421 0.56140351]
[0.48275862 0.43103448 0.4        0.43478261 0.45217391 0.47826087
 0.53043478 0.52173913 0.46956522 0.44347826]
[0.45217391 0.5        0.45614035 0.45614035 0.45614035 0.44736842
 0.46491228 0.4122807  0.49122807 0.45614035]


KeyboardInterrupt: ignored

In [ ]:
# Smile-Mouth
print('Smile-Mouth')
print("\taccuracy_rate: {:.2f}%".format(rates[0] * 100))
print("\tuncertainty_rate: {:.2f}%".format(rates[1] * 100))
print("\tinaccuracy_rate: {:.2f}%".format(rates[2] * 100))

# Smile-Nose
print('Smile-Nose')
print("\taccuracy_rate: {:.2f}%".format(rates[3] * 100))
print("\tuncertainty_rate: {:.2f}%".format(rates[4] * 100))
print("\tinaccuracy_rate: {:.2f}%".format(rates[5] * 100))

# Smile-Eye
print('Smile-Eye')
print("\taccuracy_rate: {:.2f}%".format(rates[6] * 100))
print("\tuncertainty_rate: {:.2f}%".format(rates[7] * 100))
print("\tinaccuracy_rate: {:.2f}%".format(rates[8] * 100))

# Smile-Mouth_Eye
print('Smile-Mouth_Eye')
print("\taccuracy_rate: {:.2f}%".format(rates[9] * 100))
print("\tuncertainty_rate: {:.2f}%".format(rates[10] * 100))
print("\tinaccuracy_rate: {:.2f}%".format(rates[11] * 100))

# Smile-Eye_Nose
print('Smile-Eye_Nose')
print("\taccuracy_rate: {:.2f}%".format(rates[12] * 100))
print("\tuncertainty_rate: {:.2f}%".format(rates[13] * 100))
print("\tinaccuracy_rate: {:.2f}%".format(rates[14] * 100))

# Smile-Mouth_Nose
print('Smile-Mouth_Nose')
print("\taccuracy_rate: {:.2f}%".format(rates[15] * 100))
print("\tuncertainty_rate: {:.2f}%".format(rates[16] * 100))
print("\tinaccuracy_rate: {:.2f}%".format(rates[17] * 100))


Smile-Mouth
	accuracy_rate: 70.81%
	uncertainty_rate: 15.10%
	inaccuracy_rate: 14.09%
Smile-Nose
	accuracy_rate: 72.89%
	uncertainty_rate: 13.38%
	inaccuracy_rate: 13.73%
Smile-Eye
	accuracy_rate: 68.60%
	uncertainty_rate: 17.06%
	inaccuracy_rate: 14.33%
Smile-Mouth_Eye
	accuracy_rate: 66.79%
	uncertainty_rate: 17.71%
	inaccuracy_rate: 15.50%
Smile-Eye_Nose
	accuracy_rate: 68.38%
	uncertainty_rate: 17.53%
	inaccuracy_rate: 14.09%
Smile-Mouth_Nose
	accuracy_rate: 71.03%
	uncertainty_rate: 13.45%
	inaccuracy_rate: 15.52%


All at once

In [ ]:
# Mouth Model
def get_mouth_model(includes='ns', method='svm'):
    sv_m = Model_mouth(test_size=0.2, low_number_of_images=4, kernel_type = "linear", scale_feature= True, method="rand_forest", category='smile')
    sv_m.generate_train_test_split(category='smile', includes=includes)
    sv_m.train_model(category='smile')
    return sv_m

# Nose Model
def get_nose_model(includes='ns', method='svm'):
    sv_n = Model_Nose(test_size=0.2, low_number_of_images=4, kernel_type = "linear", scale_feature= True, method="rand_forest", category='smile')
    sv_n.generate_train_test_split(category='smile', includes=includes)
    sv_n.train_model(category='smile')
    return sv_n

# Eye Model
def get_eye_model(includes='ns', method='svm'):
    sv_e = Model_eye(test_size=0.2, low_number_of_images=4, kernel_type = "linear", scale_feature= True, method="rand_forest", category='smile')
    sv_e.generate_train_test_split(category='smile', includes=includes)
    sv_e.train_model(category='smile')
    return sv_e

# Mouth_Eye Model
def get_mouth_eye_model(includes='ns', method='svm'):
    sv_me = Model_mouth_eye(test_size=0.2, low_number_of_images=4, kernel_type = "linear", scale_feature= True, method="rand_forest", category='smile')
    sv_me.generate_train_test_split(category='smile', includes=includes)
    sv_me.train_model(category='smile')
    return sv_me

# Eye_Nose Model
def get_eye_nose_model(includes='ns', method='svm'):
    sv_en = Model_eye_nose(test_size=0.2, low_number_of_images=4, kernel_type = "linear", scale_feature= True, method="rand_forest", category='smile')
    sv_en.generate_train_test_split(category='smile', includes=includes)
    sv_en.train_model(category='smile')
    return sv_en

# Mouth_Nose Model
def get_mouth_nose_model(includes='ns', method='svm'):
    sv_mn = Model_mouth_nose(test_size=0.2, low_number_of_images=4, kernel_type = "linear", scale_feature= True, method="rand_forest", category='smile')
    sv_mn.generate_train_test_split(category='smile', includes=includes)
    sv_mn.train_model(category='smile')
    return sv_mn



def task(includes='ns', method='svm', max_iter=20):
    rates = np.zeros(20)
    MAX_ITER = max_iter
    print('\r' + '-'*int(MAX_ITER*100/MAX_ITER), end=f' {0}%')
    for i in range(MAX_ITER):
        # print(i, end=' ')
        mouth_model = get_mouth_model(includes=includes, method=method)
        nose_model = get_nose_model(includes=includes, method=method)
        eye_model = get_eye_model(includes=includes, method=method)
        mouth_eye_model = get_mouth_eye_model(includes=includes, method=method)
        eye_nose_model = get_eye_nose_model(includes=includes, method=method)
        mouth_nose_model = get_mouth_nose_model(includes=includes, method=method)

        print('\r' + '*'*int((i+0.5)*100/MAX_ITER) + '-'*int((MAX_ITER-i-0.5)*100/MAX_ITER), end=f' {(i+0.5)/MAX_ITER * 100}%')


        # Smile_Mouth
        accuracy_rate, not_sure_rate, inaccuracy_rate = mouth_model.get_accuracy()
        rates[0] += accuracy_rate
        rates[1] += not_sure_rate
        rates[2] += inaccuracy_rate

        # Smile-Nose
        accuracy_rate, not_sure_rate, inaccuracy_rate = nose_model.get_accuracy()
        rates[3] += accuracy_rate
        rates[4] += not_sure_rate
        rates[5] += inaccuracy_rate

        # Smile-Eye
        accuracy_rate, not_sure_rate, inaccuracy_rate = eye_model.get_accuracy()
        rates[6] += accuracy_rate
        rates[7] += not_sure_rate
        rates[8] += inaccuracy_rate

        # Smile-Mouth_Eye
        accuracy_rate, not_sure_rate, inaccuracy_rate = mouth_eye_model.get_accuracy()
        rates[9] += accuracy_rate
        rates[10] += not_sure_rate
        rates[11] += inaccuracy_rate

        # Smile-Eye_Nose
        accuracy_rate, not_sure_rate, inaccuracy_rate = eye_nose_model.get_accuracy()
        rates[12] += accuracy_rate
        rates[13] += not_sure_rate
        rates[14] += inaccuracy_rate

        # Smile-Mouth_Nose
        accuracy_rate, not_sure_rate, inaccuracy_rate = mouth_nose_model.get_accuracy()
        rates[15] += accuracy_rate
        rates[16] += not_sure_rate
        rates[17] += inaccuracy_rate

        print('\r' + '*'*int((i+1)*100/MAX_ITER) + '-'*int((MAX_ITER-i-1)*100/MAX_ITER), end=f' {(i+1)/MAX_ITER * 100}%')

    rates = rates / MAX_ITER


    # Smile-Mouth
    print('Smile-Mouth')
    print("\taccuracy_rate: {:.2f}%".format(rates[0] * 100))
    print("\tuncertainty_rate: {:.2f}%".format(rates[1] * 100))
    print("\tinaccuracy_rate: {:.2f}%".format(rates[2] * 100))

    # Smile-Nose
    print('Smile-Nose')
    print("\taccuracy_rate: {:.2f}%".format(rates[3] * 100))
    print("\tuncertainty_rate: {:.2f}%".format(rates[4] * 100))
    print("\tinaccuracy_rate: {:.2f}%".format(rates[5] * 100))

    # Smile-Eye
    print('Smile-Eye')
    print("\taccuracy_rate: {:.2f}%".format(rates[6] * 100))
    print("\tuncertainty_rate: {:.2f}%".format(rates[7] * 100))
    print("\tinaccuracy_rate: {:.2f}%".format(rates[8] * 100))

    # Smile-Mouth_Eye
    print('Smile-Mouth_Eye')
    print("\taccuracy_rate: {:.2f}%".format(rates[9] * 100))
    print("\tuncertainty_rate: {:.2f}%".format(rates[10] * 100))
    print("\tinaccuracy_rate: {:.2f}%".format(rates[11] * 100))

    # Smile-Eye_Nose
    print('Smile-Eye_Nose')
    print("\taccuracy_rate: {:.2f}%".format(rates[12] * 100))
    print("\tuncertainty_rate: {:.2f}%".format(rates[13] * 100))
    print("\tinaccuracy_rate: {:.2f}%".format(rates[14] * 100))

    # Smile-Mouth_Nose
    print('Smile-Mouth_Nose')
    print("\taccuracy_rate: {:.2f}%".format(rates[15] * 100))
    print("\tuncertainty_rate: {:.2f}%".format(rates[16] * 100))
    print("\tinaccuracy_rate: {:.2f}%".format(rates[17] * 100))

    return mouth_model, nose_model, eye_model, mouth_eye_model, eye_nose_model, mouth_nose_model

In [ ]:
n_iter = 10 #10
iterations = [
    ['ns', 'svm', n_iter],
    # ['n', 'svm', n_iter],
    ['ns', 'rand_forest', n_iter],
    # ['n', 'rand_forest', n_iter],
    # ['ns', 'decision_tree', n_iter],
    # ['n', 'decision_tree', n_iter],
    # ['ns', 'knn', n_iter],
    # ['n', 'knn', n_iter],
    ['ns', 'lda', n_iter],
    # ['n', 'lda', n_iter],
    ['ns', 'lr', n_iter],
    # ['n', 'lr', n_iter],
    # ['ns', 'naive_bayes', n_iter],
    # ['n', 'naive_bayes', n_iter],
    # ['ns', 'gbm', 1],  # very slow
    # ['n', 'gbm', 1],   # very slow
]

for iter in iterations:
    task(iter[0], iter[1], iter[2])

---------------------------------------------------------------------------------------------------- 0%[0.45299145 0.37068966 0.54310345 0.37068966 0.42241379 0.44827586
 0.45689655 0.39655172 0.51724138 0.51724138]
[0.51724138 0.46551724 0.4137931  0.43965517 0.44827586 0.43478261
 0.4        0.47826087 0.45217391 0.48695652]
[0.43965517 0.46551724 0.38793103 0.51304348 0.54782609 0.51304348
 0.44347826 0.45217391 0.43478261 0.45217391]
[0.49565217 0.46956522 0.46956522 0.46956522 0.47368421 0.37719298
 0.40350877 0.43859649 0.53508772 0.54385965]
[0.46491228 0.42982456 0.48245614 0.42105263 0.46491228 0.33333333
 0.35964912 0.5        0.45614035 0.43362832]
[0.47008547 0.44444444 0.5042735  0.45299145 0.42735043 0.43589744
 0.48717949 0.46153846 0.35042735 0.47008547]
**********------------------------------------------------------------------------------------------ 10.0%[0.42477876 0.37168142 0.50892857 0.42857143 0.44642857 0.41964286
 0.52678571 0.5        0.47321429 0.47321429]


KeyboardInterrupt: ignored